In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [17]:
conf = SparkConf().setMaster("local").setAppName("temp")
sc.stop()
sc = SparkContext(conf = conf)

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example")  \
    .getOrCreate()

In [29]:
books = spark.read.option('compression', 'gzip').json("/Users/frankchen/Desktop/reviews_Books_5.json.gz")
books.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [30]:
metadata = spark.read.option('compression', 'gzip').json("/Users/frankchen/Desktop/metadata.json.gz")
metadata.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- description: string (nullable = true)
 |-- imUrl: string (nullable = true)
 |-- price: double (nullable = true)
 |-- related: struct (nullable = true)
 |    |-- also_bought: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- also_viewed: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- bought_together: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- buy_after_viewing: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- salesRank: struct (nullable = true)
 |    |-- Appliances: long (nullable = true)
 |    |-- Arts, Crafts & Sewing: long (nullable = true)
 |    |-- Automotive: long

In [31]:
books.createGlobalTempView("books")
metadata.createGlobalTempView("metadata")

In [32]:
spark.sql("Select asin, overall from global_temp.books ").show()

+----------+-------+
|      asin|overall|
+----------+-------+
|000100039X|    5.0|
|000100039X|    5.0|
|000100039X|    5.0|
|000100039X|    5.0|
|000100039X|    5.0|
|000100039X|    5.0|
|000100039X|    5.0|
|000100039X|    5.0|
|000100039X|    5.0|
|000100039X|    5.0|
|000100039X|    5.0|
|000100039X|    5.0|
|000100039X|    5.0|
|000100039X|    5.0|
|000100039X|    5.0|
|000100039X|    5.0|
|000100039X|    5.0|
|000100039X|    5.0|
|000100039X|    5.0|
|000100039X|    5.0|
+----------+-------+
only showing top 20 rows



In [33]:
spark.sql("Select asin, title from global_temp.metadata ").show()

+----------+--------------------+
|      asin|               title|
+----------+--------------------+
|0001048791|The Crucible: Per...|
|0000143561|Everyday Italian ...|
|0000037214|Purple Sequin Tin...|
|0000032069|Adult Ballet Tutu...|
|0000031909|Girls Ballet Tutu...|
|0000032034|Adult Ballet Tutu...|
|0000589012|Why Don't They Ju...|
|0001048775|Measure for Measu...|
|0000031852|Girls Ballet Tutu...|
|0001048236|The Sherlock Holm...|
|0000401048|The rogue of publ...|
|0001019880|Classic Soul Winn...|
|0001048813|Archer Christmas ...|
|0001148427| Sonatas - For Piano|
|0001057170|Classic Connolly ...|
|0001047566|       Hand in Glove|
|0001053396|War Poems: An Ant...|
|0000913154|The Way Things Wo...|
|0000032050|Adult Ballet Tutu...|
|0001072986|As You Like it: C...|
+----------+--------------------+
only showing top 20 rows



In [34]:
booksWithTitle = spark.sql("SELECT b.asin, b.overall, m.title FROM global_temp.books b, global_temp.metadata m"
          " where b.asin = m.asin")

In [37]:
# booksWithTitle.write.json("bookstitle.json")
booksWithTitle.coalesce(1).write.format('json').save("./booksWithTitle.json")

Py4JJavaError: An error occurred while calling o410.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.sc